In [26]:
import pandas as pd 
import numpy as np

In [27]:
df =pd.read_csv('C:/Users/Grodencia/Documents/MACHINE LEARNING/LoanPred/train_LZV4RXX.csv',index_col='loan_id')
test = pd.read_csv('C:/Users/Grodencia/Documents/MACHINE LEARNING/LoanPred/test_4zJg83n.csv',)

In [28]:
ID  = test['loan_id'].to_frame('loan_id').reset_index(drop = True)

In [29]:
Data = test.drop(columns=['loan_id'])

In [30]:
features = df.drop(columns=['loan_default'])
target = df['loan_default']

In [31]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import CountEncoder

def preprocess_pipeline(df):
    def preprocess_data(df, numerical_features, categorical_features):
        # Define the transformers for numerical and categorical features
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('scaler', StandardScaler())
        ])

        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', CountEncoder())
        ])

        # Create the ColumnTransformer
        preprocessor = ColumnTransformer(transformers=[
            ('numeric', numeric_transformer, numerical_features),
            ('categorical', categorical_transformer, categorical_features)
        ])

        # Apply preprocessing to the dataset
        preprocessed_data = preprocessor.fit_transform(df)

        # Get the column names after preprocessing
        transformed_cols = numerical_features + categorical_features
        transformed_data = pd.DataFrame(preprocessed_data, columns=transformed_cols)

        return transformed_data

    def select_features(df):
        # Select numerical features using select_dtypes
        numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

        # Select categorical features using select_dtypes
        categorical_features = df.select_dtypes(include='object').columns.tolist()

        return numerical_features, categorical_features

    # Call the select_features function to determine the numerical and categorical features
    numerical_features, categorical_features = select_features(df)

    # Call the preprocess_data function to preprocess the dataset
    preprocessed_data = preprocess_data(df, numerical_features, categorical_features)

    return preprocessed_data




In [32]:
# Assuming you have defined df as the test dataset
X = preprocess_pipeline(features)
X

,age,education,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none,proof_submitted
0,-1.198943,-1.168616,-1.062009,-0.444709,-0.155950,0.287265,-0.116037,5931.0
1,1.568892,-1.168616,0.435246,-0.395640,0.575106,0.287265,-0.116037,5931.0
2,-0.803538,0.855713,-0.136308,-0.420594,-0.521478,-0.626344,-0.116037,707.0
3,-1.067142,-1.168616,0.555405,0.068186,-0.521478,-0.626344,-0.116037,5931.0
4,-0.935340,-1.168616,3.181030,4.595834,-0.521478,-0.626344,-0.116037,5931.0
...,...,...,...,...,...,...,...,...
6995,1.173487,0.855713,1.786309,1.594783,-0.521478,-0.626344,-0.116037,5931.0
6996,0.646280,0.855713,0.118509,-0.584024,0.209578,1.200873,-0.116037,5931.0
6997,1.437090,0.855713,-0.238442,-0.913682,5.875258,4.398503,-0.116037,5931.0
6998,0.382677,-1.168616,-0.057724,-0.570876,-0.521478,-0.626344,-0.116037,5931.0


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [34]:
#Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

In [35]:


def train_models(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "Gradient Boosting": GradientBoostingClassifier()
    }

    trained_models = {}
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        trained_models[model_name] = model

    return trained_models

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    # Evaluate the Model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    results = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC AUC Score": roc_auc
    }

    return results


In [36]:

# Train the models
trained_models = train_models(X_train, y_train)


In [37]:
# Select a particular model
selected_model_name = "Logistic Regression"
selected_model = trained_models[selected_model_name]

# Evaluate the selected model
selected_model_results = evaluate_model(selected_model, X_test, y_test)


c:\Users\Grodencia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
# Print the results for the selected model
print(f"Metrics for {selected_model_name}:")
for metric_name, value in selected_model_results.items():
    print(f"{metric_name}: {value}")

Metrics for Logistic Regression:
Accuracy: 0.6207142857142857
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
ROC AUC Score: 0.5


In [40]:
preprocessed_new_data = preprocess_pipeline(Data)


In [43]:
predictions = pd.Series(selected_model.predict(preprocessed_new_data))


In [44]:
predictions

0       0
1       0
2       0
3       0
4       0
       ..
2995    0
2996    0
2997    0
2998    0
2999    0
Length: 3000, dtype: int64

In [45]:
ld = predictions.to_frame('loan_default').reset_index(drop = True)

In [46]:
df_LG = pd.concat([ID, ld], axis = 1)
print(df_LG )

      loan_id  loan_default
0        7001             0
1        7002             0
2        7003             0
3        7004             0
4        7005             0
...       ...           ...
2995     9996             0
2996     9997             0
2997     9998             0
2998     9999             0
2999    10000             0

[3000 rows x 2 columns]
